# Blue Brain BioExplorer
![](../../bioexplorer_banner.png)

In [ ]:
from bioexplorer import BioExplorer
from mediamaker import MovieMaker
import os

hostname = 'localhost:5000'
resource_folder = '../../../tests/test_files/'
    
be = BioExplorer(hostname)
core = be.core_api()
mm = MovieMaker(be)
version = be.version()
print('Version ' + version)

sequence_name = 'high_glucose'

## Data loading

In [ ]:
'''Reset current scene'''
be.reset()

In [ ]:
'''Accelerate loading by not showing models as they are loaded'''
status = be.set_general_settings(
    logging_enabled=True,
    model_visibility_on_creation=False)

In [ ]:
cache_folder = resource_folder + 'caches/' + version + '/'

In [ ]:
cache_filename = cache_folder + sequence_name + '_scenario.bioexplorer'
print('Loading ' + cache_filename)
dummy = core.add_model(cache_filename)

In [ ]:
nb_models = len(core.scene.models)
if nb_models != 180:
    print('ERROR: Not all models have been loaded!!! Expected 180, found ' + str(nb_models))
else:
    print('All good!')

In [ ]:
'''Now we can show the models'''
status = be.set_models_visibility(True)

## Rendering settings

In [ ]:
status = core.set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer', head_light=False,
    samples_per_pixel=1, subsampling=1, max_accum_frames=128)
params = be.core_api().BioExplorerRendererParams()
params.exposure = 1.0
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 5000
params.shadows = 1.0
params.soft_shadows = 0.02
params.fog_start = 1000
params.fog_thickness = 300
params.max_bounces = 1
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

status = core.clear_lights()
status = core.add_light_directional(
    angularDiameter=0.5, color=[1,1,1], direction=[-0.7,-0.4,-1],
    intensity=1.0, is_visible=False
)

## Image settings

In [ ]:
image_size = [3840, 2160]
image_samples_per_pixel = 64

output_folder = resource_folder + 'frames/' + \
                version + '/' + sequence_name + '/'
print(output_folder)

command_line='mkdir -p ' + output_folder
os.system(command_line)
command_line='ls ' + output_folder

if os.system(command_line) != 0:
    print('ERROR: Failed to create output folder')

## Snapshot

In [ ]:
def set_all_materials_to_grey():
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)
        palette = list()
        for p in range(nb_materials):
            palette.append([0.25,0.25,0.25])
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids, 
            diffuse_colors=palette, specular_colors=palette)

In [ ]:
def apply_glucose_colors():
    import seaborn as sns
    shading_mode = be.SHADING_MODE_BASIC
    specular_exponent = 50
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)

        if be.NAME_GLUCOSE in model_name and 'Surfactant' in model_name:
            print(model_name)
            palette = sns.color_palette('OrRd', nb_materials)
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids, palette=palette)
        if be.NAME_LACTOFERRIN in model_name or \
           be.NAME_SURFACTANT_HEAD in model_name or \
           be.NAME_DEFENSIN in model_name:
            print(model_name)
            palette = list()
            for p in range(nb_materials):
                palette.append([0.25,0.25,0.25])
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids,
                palette=palette)

### Frame 1100 - Surfactant head

In [ ]:
status = core.set_camera(
    orientation=[-0.024, -0.139, -0.003, 0.989],
    position=[38.749, 35.228, 5.536],
    target=[39.025, 35.179, 4.576]
)

In [ ]:
set_all_materials_to_grey()
apply_glucose_colors()

In [ ]:
mm.create_snapshot(
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/01100.png')